## **Ejemplos: Agentes Multi-IA para reservar hotel**

En el mundo acelerado de hoy, planificar un viaje de negocios implica más que solo reservar un vuelo y una habitación de hotel. Requiere un nivel de coordinación y eficiencia que puede ser difícil de lograr. Aquí es donde entran en juego los Agentes Multi-IA, revolucionando la forma en que gestionamos nuestras necesidades de viaje.

Imagínese tener un equipo de agentes inteligentes a su disposición, trabajando juntos para manejar cada aspecto de su viaje con precisión y facilidad. Con nuestra avanzada tecnología de IA, hemos creado agentes especializados para la reserva de servicios y la organización del itinerario, asegurando una experiencia de viaje fluida y sin estrés.

Este es un escenario básico. Al planificar un viaje de negocios, necesitamos consultar con un agente de viajes de negocios para obtener información sobre boletos aéreos, información de hotel, etc. A través de los Agentes de IA, podemos construir agentes para la reserva de servicios y agentes para la organización del itinerario que colaboren y mejoren el nivel de inteligencia.


# Inicializar el Servicio Azure AI Agent y obtener información de configuración desde **.env**

### **.env** 

Crea un archivo .env 

**.env** contiene la cadena de conexión del Servicio Azure AI Agent, el modelo utilizado por AOAI, y el correspondiente servicio API de Google API Search, ENDPOINT, etc.

- **AZURE_AI_AGENT_MODEL_DEPLOYMENT_NAME** = "Tu Nombre de Despliegue del Modelo del Servicio Azure AI Agent"

[**NOTA**] Necesitarás un modelo con límite de tasa de 100,000 (Tokens por minuto) Límite de tasa de 600 (Solicitudes por minuto)

  Puedes obtener el modelo en Microsoft Foundry - Modelo y Endpoint. 


- **AZURE_AI_AGENT_PROJECT_CONNECTION_STRING** = "Tu Cadena de Conexión del Proyecto del Servicio Azure AI Agent"

  Puedes obtener la cadena de conexión del proyecto en la vista general de tu proyecto en la pantalla del Portal AI Foundry.

- **SERPAPI_SEARCH_API_KEY** = "Tu Clave API de Búsqueda SERPAPI"
- **SERPAPI_SEARCH_ENDPOINT** = "Tu Endpoint de Búsqueda SERPAPI"

Para obtener el Nombre de Despliegue del Modelo y la Cadena de Conexión del Proyecto del Servicio Azure AI Agent, necesitas crear el Servicio Azure AI Agent. Se recomienda usar [esta plantilla](https://portal.azure.com/#create/Microsoft.Template/uri/https%3A%2F%2Fraw.githubusercontent.com%2Ffosteramanda%2Fazure-agent-quickstart-templates%2Frefs%2Fheads%2Fmaster%2Fquickstarts%2Fmicrosoft.azure-ai-agent-service%2Fstandard-agent%2Fazuredeploy.json) para crearlo directamente (***Nota:*** El Servicio Azure AI Agent actualmente está configurado en una región limitada. Se recomienda que consultes [este enlace](https://learn.microsoft.com/en-us/azure/ai-services/agents/concepts/model-region-support) para configurar la región)

El agente necesita acceder a SERPAPI. Se recomienda registrarse usando [este enlace](https://serpapi.com/searches). Después del registro, podrás obtener una Clave API única y un ENDPOINT


# Iniciar sesión en Azure 

Ahora debes iniciar sesión en Azure. Abre una terminal en VScode y ejecuta el comando `az login`


# Configuración 

Para ejecutar este cuaderno, deberá instalar las siguientes bibliotecas. Aquí hay una lista de las bibliotecas requeridas y los comandos correspondientes para instalar con pip:

azure-identity: Para la autenticación de Azure.  
requests: Para realizar solicitudes HTTP.  
semantic-kernel: Para el marco de kernel semántico (suponiendo que esta es una biblioteca personalizada o específica, es posible que deba instalarla desde una fuente o repositorio específico).


In [ ]:
!pip install azure-identity
!pip install requests
!pip install semantic-kernel
!pip install --upgrade semantic_kernel
!pip install azure-cli

# Explicación: 
import asyncio: Esto importa el módulo asyncio, que proporciona soporte para programación asíncrona en Python. Permite escribir código concurrente usando la sintaxis async y await.
from typing import Annotated: Esto importa el tipo Annotated del módulo typing. Annotated se usa para agregar metadatos a las pistas de tipo, lo que puede ser útil para diversos propósitos como validación, documentación o herramientas


In [ ]:
import asyncio,os
from typing import Annotated

# Explicación:
Al usar from dotenv import load_dotenv y load_dotenv(), puedes gestionar fácilmente configuraciones y información sensible (como claves API y URLs de bases de datos) en un archivo .env, manteniéndolos separados de tu código fuente y haciendo que tu aplicación sea más segura y fácil de configurar.


In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Explicación:

Instrucción de importación: from azure.identity.aio import DefaultAzureCredential: Esto importa la clase DefaultAzureCredential desde el módulo azure.identity.aio. La parte aio del nombre del módulo indica que está diseñado para operaciones asincrónicas.

Propósito de DefaultAzureCredential: La clase DefaultAzureCredential es parte del SDK de Azure para Python. Proporciona una forma predeterminada de autenticarse con los servicios de Azure. Intenta autenticarse utilizando múltiples métodos en un orden específico, como variables de entorno, identidad administrada y credenciales de Azure CLI.

Operaciones asincrónicas: El módulo aio indica que la clase DefaultAzureCredential soporta operaciones asincrónicas. Esto significa que puedes usarla con asyncio para realizar solicitudes de autenticación no bloqueantes.


In [ ]:
from azure.identity.aio import DefaultAzureCredential

# Explicación:
Importa varios módulos y clases del paquete semantic_kernel. Aquí hay un desglose de cada importación:

AgentGroupChat desde semantic_kernel.agents: Esta clase maneja funcionalidades relacionadas con el chat grupal para agentes de IA. AzureAIAgent y AzureAIAgentSettings desde semantic_kernel.agents.azure_ai

AzureAIAgent: Esta clase se usa para crear y administrar agentes de IA que utilizan servicios de Azure AI.

AzureAIAgentSettings: Esta clase se usa para configurar los ajustes del AzureAIAgent. TerminationStrategy desde semantic_kernel.agents.strategies.termination.termination_strategy:

Esta clase define estrategias para terminar la ejecución de agentes de IA bajo ciertas condiciones. ChatMessageContent desde semantic_kernel.contents.chat_message_content:

Esta clase se usa para manejar el contenido de mensajes de chat.
AuthorRole desde semantic_kernel.contents.utils.author_role:

Esta clase define diferentes roles para los autores en el contexto de los mensajes de chat.

kernel_function desde semantic_kernel.functions.kernel_function_decorator: Este decorador se usa para definir funciones del kernel, que son funciones que pueden ejecutarse dentro del marco de trabajo del semantic kernel.
Estas importaciones configuran los componentes necesarios para crear y administrar agentes de IA que puedan interactuar en un entorno de chat grupal, posiblemente para tareas como reservar hoteles o actividades similares.


In [ ]:
from semantic_kernel.agents import AgentGroupChat
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentSettings
from semantic_kernel.agents.strategies.termination.termination_strategy import TerminationStrategy
from semantic_kernel.contents import ChatMessageContent
from semantic_kernel.contents import AuthorRole
from semantic_kernel.functions.kernel_function_decorator import kernel_function

# Explicación:
A continuación importamos la clase CodeInterpreterTool del módulo azure.ai.projects.models.

CodeInterpreterTool: Esta clase es parte del SDK de Azure AI y se utiliza para interpretar y ejecutar código dentro del contexto de proyectos de IA. Proporciona funcionalidades para ejecutar fragmentos de código, analizar código o integrar la ejecución de código dentro de flujos de trabajo de IA.
Esta importación configura el componente necesario para utilizar CodeInterpreterTool en su proyecto, lo que puede ser útil para tareas que implican interpretar y ejecutar código de forma dinámica.


In [ ]:
from azure.ai.projects.models import CodeInterpreterTool

# Explicación: 
La clase ApprovalTerminationStrategy proporciona una estrategia específica para terminar la operación de un agente de IA. El agente finalizará si el último mensaje en su historial de interacción contiene la palabra "saved". Esto podría ser útil en escenarios donde se considera que la tarea del agente está completa una vez que recibe la confirmación de que algo ha sido "saved". Definir el método de interacción. Después de que el plan de reserva sea guardado, puede detenerse al recibir la señal de guardado.


In [ ]:
class ApprovalTerminationStrategy(TerminationStrategy):
    """A strategy for determining when an agent should terminate."""

    async def should_agent_terminate(self, agent, history):
        """Check if the agent should terminate."""
        return "saved" in history[-1].content.lower()

# Explicación:

La línea de código inicializa un objeto AzureAIAgentSettings con configuraciones predeterminadas o predefinidas llamando al método create(). Este objeto de configuración (ai_agent_settings) luego puede usarse para configurar y administrar una instancia de AzureAIAgent.


In [ ]:
ai_agent_settings = AzureAIAgentSettings.create()

# Explicación:
Al importar la biblioteca requests, puedes realizar fácilmente solicitudes HTTP e interactuar con servicios web en tu código Python.


In [ ]:
import requests

# Explicación:
Esta es una variable que almacena la clave API para acceder a un servicio de API de SERP (página de resultados de motor de búsqueda). Una clave API es un identificador único usado para autenticar solicitudes asociadas con tu cuenta.

'GOOGLE_SEARCH_API_KEY': Esta es una cadena de marcador de posición. Debes reemplazar ''GOOGLE_SEARCH_API_KEY' con tu clave API real de SERP.

Propósito: El propósito de esta línea es almacenar la clave API en una variable para que pueda ser usada para autenticar solicitudes al servicio de API de SERP. La clave API es necesaria para acceder al servicio y realizar búsquedas.

Cómo obtener una clave API de SERP: Para obtener una clave API de SERP, sigue estos pasos generales en https://serpapi.com (los pasos exactos pueden variar dependiendo del servicio específico de API de SERP que estés usando):

Elige un servicio de API de SERP: Hay varios servicios de API de SERP disponibles, como SerpAPI, Google Custom Search JSON API, y otros. Elige el que mejor se adapte a tus necesidades.

Regístrate para obtener una cuenta:

Ve al sitio web del servicio de API de SERP elegido https://www.serpapi.com y regístrate para una cuenta. Es posible que necesites proporcionar alguna información básica y verificar tu dirección de correo electrónico.

Crea una clave API:

Después de registrarte, inicia sesión en tu cuenta y navega a la sección o panel del API. Busca una opción para crear o generar una nueva clave API.
Copia la clave API:

Una vez que la clave API esté generada, cópiala. Esta clave será usada para autenticar tus solicitudes al servicio de API de SERP.
Reemplaza el marcador de posición:

Reemplaza el marcador de posición en tu archivo .env


In [ ]:
SERPAPI_SEARCH_API_KEY=os.getenv('SERPAPI_SEARCH_API_KEY')

In [ ]:
SERPAPI_SEARCH_ENDPOINT = os.getenv('SERPAPI_SEARCH_ENDPOINT')

# Explicación:
La clase BookingPlugin proporciona métodos para reservar hoteles y vuelos utilizando la API de búsqueda de Google de Serpapi.com. Construye los parámetros necesarios, envía solicitudes a la API y procesa las respuestas para devolver información relevante sobre las reservas. La clave de API (SERPAPI_SEARCH_API_KEY) y el punto final (SERPAPI_SEARCH_ENDPOINT) se utilizan para autenticar y enviar solicitudes a la API de búsqueda de Google.


In [ ]:
# Define Booking Plugin
class BookingPlugin:
    """Booking Plugin for customers"""
    @kernel_function(description="booking hotel")
    def booking_hotel(self,query: Annotated[str, "The name of the city"], check_in_date: Annotated[str, "Hotel Check-in Time"], check_out_date: Annotated[str, "Hotel Check-in Time"])-> Annotated[str, "Return the result of booking hotel infomation"]:

        params = {
            "engine": "google_hotels",
            "q": query,
            "check_in_date": check_in_date,
            "check_out_date": check_out_date,
            "adults": "2",
            "currency": "USD",
            "gl": "us",
            "hl": "en",
            "api_key": SERPAPI_SEARCH_API_KEY
        }

        response = requests.get(SERPAPI_SEARCH_ENDPOINT, params=params)
        if response.status_code == 200:
            response = response.json()
            return response["properties"]
        else:
            return None

    
    @kernel_function(description="booking fight")
    def  booking_fight(self,origin: Annotated[str, "The name of Departure"], destination: Annotated[str, "The name of Destination"], outbound_date: Annotated[str, "The date of outbound"], return_date: Annotated[str, "The date of Return_date"])-> Annotated[str, "Return the result of booking fight infomation"]:
        
        go_params = {
            "engine": "google_flights",   
            "departure_id": origin,
            "arrival_id": destination,
            "outbound_date": outbound_date,
            "return_date": return_date,  
            "currency": "USD",
            "hl": "en",
            "api_key": SERPAPI_SEARCH_API_KEY  
        }

        print(go_params)

        go_response = requests.get(SERPAPI_SEARCH_ENDPOINT, params=go_params)


        result = ''

        if go_response.status_code == 200:
            response = go_response.json()

            result += "# outbound \n " + str(response)
        else:
            print('error!!!')
            # return None

        
        back_params = {
            "engine": "google_flights",   
            "departure_id": destination,
            "arrival_id": origin,
            "outbound_date": return_date,
            "return_date": return_date,  
            "currency": "USD",
            "hl": "en",
            "api_key": SERPAPI_SEARCH_API_KEY  
        }


        print(back_params)


        back_response = requests.get(SERPAPI_SEARCH_ENDPOINT, params=back_params)



        if back_response.status_code == 200:
            response = back_response.json()

            result += "\n # return \n"  + str(response)

        else:
            print('error!!!')
            # return None
        
        print(result)

        return result

        


# Explicación:
La clase SavePlugin proporciona un método saving_plan para guardar planes de viaje utilizando los servicios de Azure AI. Configura las credenciales de Azure, crea un agente de IA, procesa las entradas del usuario para generar y guardar el contenido del plan de viaje, y maneja las operaciones de guardado de archivos y limpieza. El método devuelve "Guardado" al completar con éxito.


In [ ]:
class SavePlugin:
    """Save Plugin for customers"""
    @kernel_function(description="saving plan")
    async def saving_plan(self,tripplan: Annotated[str, "The content of trip plan"])-> Annotated[str, "Return status of save content"]:

        async with (
            DefaultAzureCredential() as creds,
            AzureAIAgent.create_client(
                credential=creds,
                conn_str=ai_agent_settings.project_connection_string.get_secret_value(),
            ) as client,
        ):

            code_interpreter = CodeInterpreterTool()
            
            agent_definition = await client.agents.create_agent(
                model=ai_agent_settings.model_deployment_name,
                tools=code_interpreter.definitions,
                tool_resources=code_interpreter.resources,
            )


            agent = AzureAIAgent(
                client=client,
                definition=agent_definition,
            )

            thread = await client.agents.create_thread()


            user_inputs = [
                """
            
                        You are my Python programming assistant. Generate code,save """+ tripplan +
                        
                    """    
                        and execute it according to the following requirements

                        1. Save blog content to trip-{YYMMDDHHMMSS}.md

                        2. give me the download this file link
                    """
            ]



            try:
                for user_input in user_inputs:
                    # Add the user input as a chat message
                    await agent.add_chat_message(
                        thread_id=thread.id, message=ChatMessageContent(role=AuthorRole.USER, content=user_input)
                    )
                    print(f"# User: '{user_input}'")
                    # Invoke the agent for the specified thread
                    async for content in agent.invoke(thread_id=thread.id):
                        if content.role != AuthorRole.TOOL:
                            print(f"# Agent: {content.content}")

                    
                    messages = await client.agents.list_messages(thread_id=thread.id)

                    # OpenAIPageableListOfThreadMessage
                    # OpenAIPageableListOfThreadMessage


                    for file_path_annotation in messages.file_path_annotations:

                            file_name = os.path.basename(file_path_annotation.text)

                            await client.agents.save_file(file_id=file_path_annotation.file_path.file_id, file_name=file_name,target_dir="./trip")

                    
            finally:
                await client.agents.delete_thread(thread.id)
                await client.agents.delete_agent(agent.id)


        return "Saved"

# Explicación:
Este código configura agentes de Azure AI para gestionar la reserva de vuelos y hoteles, y guardar planes de viaje basándose en las entradas del usuario. Utiliza credenciales de Azure para crear y configurar los agentes, procesa las entradas del usuario a través de un chat grupal, y asegura la limpieza adecuada después de completar las tareas. Los agentes usan plugins específicos (BookingPlugin y SavePlugin) para realizar sus respectivas tareas.


In [ ]:
async with (
    DefaultAzureCredential() as creds,
    AzureAIAgent.create_client(
        credential=creds,
        conn_str=ai_agent_settings.project_connection_string.get_secret_value(),
    ) as client,
):
    BOOKING_AGENT_NAME = "BookingAgent"
    BOOKING_AGENT_INSTRUCTIONS = """
    You are a booking agent. Help me book flights or hotels.

    Thought: Please understand the user's intention and confirm whether to use the reservation system to complete the task.

    Actions:
    - For flight bookings, convert the departure and destination names into airport codes.
    - Use the appropriate API for hotel or flight bookings. Verify that all necessary parameters are available. If any parameters are missing, ask the user to provide them. If all parameters are complete, call the corresponding function.
    - If the task is not related to hotel or flight booking, respond with the final answer only.
    - Output the results using a markdown table:
      - For flight bookings, output separate outbound and return contents in the order of:
        Departure Airport | Airline | Flight Number | Departure Time | Arrival Airport | Arrival Time | Duration | Airplane | Travel Class | Price (USD) | Legroom | Extensions | Carbon Emissions (kg).
      - For hotel bookings, output in the order of:
        Property Name | Property Description | Check-in Time | Check-out Time | Prices | Nearby Places | Hotel Class | GPS Coordinates.
    """

    SAVE_AGENT_NAME = "SaveAgent"
    SAVE_AGENT_INSTRUCTIONS = """
    You are a save tool agent. Help me to save the trip plan.
    """

    # Create agent definition
    booking_agent_definition = await client.agents.create_agent(
        model=ai_agent_settings.model_deployment_name,
        name=BOOKING_AGENT_NAME,
        instructions=BOOKING_AGENT_INSTRUCTIONS,
    )

    # Create the AzureAI Agent
    booking_agent = AzureAIAgent(
        client=client,
        definition=booking_agent_definition,
        # Optionally configure polling options
        # polling_options=RunPollingOptions(run_polling_interval=timedelta(seconds=1)),
    )

    # Add the sample plugin to the kernel
    booking_agent.kernel.add_plugin(BookingPlugin(), plugin_name="booking")

    # Create agent definition
    save_agent_definition = await client.agents.create_agent(
        model=ai_agent_settings.model_deployment_name,
        name=SAVE_AGENT_NAME,
        instructions=SAVE_AGENT_INSTRUCTIONS
    )

    # Create the AzureAI Agent
    save_agent = AzureAIAgent(
        client=client,
        definition=save_agent_definition,
    )

    save_agent.kernel.add_plugin(SavePlugin(), plugin_name="saving")

    user_inputs = [
        "I have a business trip from London to New York in Feb 20 2025 to Feb 27 2025 ,help me to book a hotel and fight tickets and save it"
    ]

    chat = AgentGroupChat(
        agents=[booking_agent, save_agent],
        termination_strategy=ApprovalTerminationStrategy(agents=[save_agent], maximum_iterations=10),
    )

    try:
        for user_input in user_inputs:
            # Add the user input as a chat message
            await chat.add_chat_message(
                ChatMessageContent(role=AuthorRole.USER, content=user_input)
            )
            print(f"# User: '{user_input}'")

            async for content in chat.invoke():
                print(f"# {content.role} - {content.name or '*'}: '{content.content}'")

            print(f"# IS COMPLETE: {chat.is_complete}")

            print("*" * 60)
            print("Chat History (In Descending Order):\n")
            async for message in chat.get_chat_messages(agent=save_agent):
                print(f"# {message.role} - {message.name or '*'}: '{message.content}'")
    finally:
        await chat.reset()
        await client.agents.delete_agent(save_agent.id)
        await client.agents.delete_agent(booking_agent.id)


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Aviso legal**:  
Este documento ha sido traducido utilizando el servicio de traducción por IA [Co-op Translator](https://github.com/Azure/co-op-translator). Aunque nos esforzamos por la precisión, tenga en cuenta que las traducciones automatizadas pueden contener errores o inexactitudes. El documento original en su idioma nativo debe considerarse la fuente autorizada. Para información crítica, se recomienda la traducción profesional humana. No nos hacemos responsables de malentendidos o interpretaciones erróneas derivadas del uso de esta traducción.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
